In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../utilities')
from basic_utilities import *
from model_utilities import *
from pandas.plotting import scatter_matrix
from ml_utilities import *
import scipy.stats as stats
import pdb
import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import pickle
sys.path.append('../Data_prep')
from data_config import *
sys.path.append('../../../../infrastructure/tools')
from feature_engineering import feature_engineering
from utilities import utilities
import re
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pdb

In [2]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import torch
from sklearn.metrics import roc_auc_score

In [3]:
from shap import KernelExplainer

In [4]:
train_sample = pd.read_csv('../../../data/processed_data/train_sample_segment0_logit.csv')

In [5]:
train_sample.shape

(23973, 135)

In [6]:
get_knots = read_yaml_file('../Data_prep/1d_knots_segment0.yaml')['get_knots']

In [7]:
knots_vars = []
for var in get_knots:
    matching_strings = [s for s in train_sample.columns.tolist() if re.search(f"{var}_", s)]
    knots_vars += matching_strings

In [8]:
get_dummy = read_yaml_file('../Data_prep/dummy_grouping_segment0_draft.yaml')['get_dummies']

In [9]:
dummy_vars = []
for var in get_dummy:
    matching_strings = [s for s in train_sample.columns.tolist() if re.search(f"{var}_", s)]
    dummy_vars += matching_strings

In [10]:
binning_features = read_yaml_file('../Data_prep/binning_features_segment0.yaml')['binning_features']

In [11]:
binning_features = [f.split(',')[0] for f in binning_features]

In [12]:
bin_vars = []
for var in binning_features:
    matching_strings = [s for s in train_sample.columns.tolist() if re.search(f"{var}_bin:", s)]
    bin_vars += matching_strings

In [13]:
bin_vars

[]

In [14]:
spec1 = knots_vars + dummy_vars + bin_vars
spec1 = sorted(list(set(spec1)))

In [15]:
Xtrain = train_sample[spec1].astype('float')
ytrain = train_sample['fraud_bool']
weights = train_sample['sample_weight']

In [16]:
model = TabNetClassifier()

model.fit(
    X_train=Xtrain.values,
    y_train=ytrain,
    weights=weights,
)

epoch 0  | loss: 0.18903 |  0:00:01s
epoch 1  | loss: 0.08815 |  0:00:02s
epoch 2  | loss: 0.08644 |  0:00:03s
epoch 3  | loss: 0.0819  |  0:00:04s
epoch 4  | loss: 0.08258 |  0:00:05s
epoch 5  | loss: 0.07653 |  0:00:06s
epoch 6  | loss: 0.07535 |  0:00:07s
epoch 7  | loss: 0.08442 |  0:00:08s
epoch 8  | loss: 0.08245 |  0:00:09s
epoch 9  | loss: 0.08138 |  0:00:10s
epoch 10 | loss: 0.07539 |  0:00:11s
epoch 11 | loss: 0.07835 |  0:00:12s
epoch 12 | loss: 0.07989 |  0:00:13s
epoch 13 | loss: 0.08366 |  0:00:14s
epoch 14 | loss: 0.07552 |  0:00:15s
epoch 15 | loss: 0.07235 |  0:00:16s
epoch 16 | loss: 0.07878 |  0:00:17s
epoch 17 | loss: 0.07827 |  0:00:18s
epoch 18 | loss: 0.07281 |  0:00:19s
epoch 19 | loss: 0.07229 |  0:00:21s
epoch 20 | loss: 0.07133 |  0:00:22s
epoch 21 | loss: 0.07799 |  0:00:23s
epoch 22 | loss: 0.0751  |  0:00:24s
epoch 23 | loss: 0.0713  |  0:00:25s
epoch 24 | loss: 0.07221 |  0:00:26s
epoch 25 | loss: 0.07405 |  0:00:27s
epoch 26 | loss: 0.07611 |  0:00:28s
e

In [17]:
predictions = model.predict(Xtrain.values)

In [18]:

accuracy = accuracy_score(ytrain, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8520418804488383


In [19]:
auroc = roc_auc_score(ytrain, predictions)
print(f"AUROC score: {auroc}")

AUROC score: 0.5032212885154062


In [20]:

# explainer = KernelExplainer(model.predict_proba, Xtrain.values) 

# # Calculate SHAP values for a subset of the test data (e.g., first 100 rows)
# shap_values = explainer.shap_values(Xtrain.values[:100])

# # Visualize the explanation for a single instance (e.g., the first instance)


In [21]:
# shap.initjs()
# shap.force_plot(explainer.expected_value[1], shap_values[1][0,:], Xtrain.values.iloc[0,:])

# # Generate a summary plot to show feature importance across the dataset
# shap.summary_plot(shap_values, Xtrain.values) 